# MGP and MI exchange volumes
The dataset *GME_MGP_MI_QUANTITA* refers to the total volume traded (both sell and buy) for each hours in the MGP and MI markets.

In [1]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])

volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,FLOWDATE,FLOWHOUR,MARKET,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
79446,20190101,1,MI1,1207.748,858.713,83.695,68.004,827.752,8.921,44.549,...,0.000,0.0,0.000,NaN,0.0,NaN,NaN,NaN,NaN,2018-12-31 15:10:52.263
79458,20190101,1,MGP,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,...,1843.786,0.0,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
79504,20190101,1,MI2,244.328,403.747,8.931,49.858,72.777,4.295,32.767,...,4.661,0.0,0.000,0.0,0.0,NaN,NaN,412.328,412.328,2018-12-31 16:41:34.837
79447,20190101,2,MI1,1262.958,913.156,77.863,104.702,896.259,5.944,27.843,...,0.356,0.0,0.000,NaN,0.0,NaN,NaN,NaN,NaN,2018-12-31 15:10:52.267
79459,20190101,2,MGP,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,...,1845.050,0.0,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,20241127,20,MGP,41948.262,42783.762,3454.943,7369.242,23603.056,1046.867,2283.798,...,NaN,0.0,3877.000,0.0,0.0,412.0,2981.0,39390.746,42283.762,2024-11-26 13:10:41.150
1279,20241127,21,MGP,39561.819,40847.819,3246.019,7029.584,22076.427,1031.956,2177.890,...,NaN,0.0,3877.000,0.0,0.0,412.0,2981.0,37454.819,40347.819,2024-11-26 13:10:41.150
1280,20241127,22,MGP,36556.981,37070.981,2967.479,6519.564,20541.635,963.880,2022.610,...,NaN,0.0,3384.773,203.6,0.0,363.0,2632.0,34062.006,36556.981,2024-11-26 13:10:41.150
1281,20241127,23,MGP,32718.374,33095.211,2637.343,5898.915,18502.467,876.329,1833.260,...,NaN,0.0,3423.359,711.0,0.0,363.0,2632.0,30100.211,32718.374,2024-11-26 13:10:41.150


In [2]:
market_labels = volumes['MARKET'].unique()

print(market_labels)

['MI1' 'MGP' 'MI2' 'MI3' 'MI4' 'MI5' 'MI6' 'MI7']


In [3]:
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

In [4]:
mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-27 19:00:00,41948.262,42783.762,3454.943,7369.242,23603.056,1046.867,2283.798,2506.305,0.0,None,...,NaN,0.0,3877.000,0.0,0.0,412.0,2981.0,39390.746,42283.762,2024-11-26 13:10:41.150
2024-11-27 20:00:00,39561.819,40847.819,3246.019,7029.584,22076.427,1031.956,2177.890,2449.015,0.0,None,...,NaN,0.0,3877.000,0.0,0.0,412.0,2981.0,37454.819,40347.819,2024-11-26 13:10:41.150
2024-11-27 21:00:00,36556.981,37070.981,2967.479,6519.564,20541.635,963.880,2022.610,2258.728,0.0,None,...,NaN,0.0,3384.773,203.6,0.0,363.0,2632.0,34062.006,36556.981,2024-11-26 13:10:41.150


In [5]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mgp_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-11-26 18:00:00,24173.480,22049.563,2024-11-25 13:10:39.207
2024-11-26 19:00:00,23498.975,21401.593,2024-11-25 13:10:39.207
2024-11-26 20:00:00,22051.242,19366.281,2024-11-25 13:10:39.207
2024-11-26 21:00:00,20417.280,17087.902,2024-11-25 13:10:39.207
2024-11-26 22:00:00,18282.654,14935.046,2024-11-25 13:10:39.207
2024-11-26 23:00:00,16961.695,12488.875,2024-11-25 13:10:39.207
2024-11-27 00:00:00,15906.283,11987.296,2024-11-26 13:10:41.140
2024-11-27 01:00:00,15249.590,11124.072,2024-11-26 13:10:41.140
2024-11-27 02:00:00,14937.318,10853.303,2024-11-26 13:10:41.140


In [6]:
mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])

In [7]:
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi1_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-11-25 18:00:00,448.393,1107.513,2024-11-25 18:22:16.607
2024-11-25 19:00:00,500.019,600.887,2024-11-25 18:22:16.607
2024-11-25 20:00:00,450.486,892.659,2024-11-25 18:22:16.607
2024-11-25 21:00:00,523.453,756.262,2024-11-25 18:22:16.607
2024-11-25 22:00:00,600.882,967.686,2024-11-25 18:22:16.607
2024-11-25 23:00:00,525.134,484.039,2024-11-25 18:22:16.607
2024-11-26 00:00:00,909.864,537.931,2024-11-26 18:22:17.717
2024-11-26 01:00:00,804.302,542.738,2024-11-26 18:22:17.720
2024-11-26 02:00:00,678.417,444.721,2024-11-26 18:22:17.720


In [8]:
mi2_volumes = volumes[volumes['MARKET'] == 'MI2']
mi2_volumes = mi2_volumes.drop(columns=['MARKET'])

In [9]:
mi2_volumes_nord = mi2_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi2_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-11-25 18:00:00,224.693,271.273,2024-11-25 18:22:20.657
2024-11-25 19:00:00,196.095,257.614,2024-11-25 18:22:20.657
2024-11-25 20:00:00,216.654,232.211,2024-11-25 18:22:20.657
2024-11-25 21:00:00,208.997,252.573,2024-11-25 18:22:20.657
2024-11-25 22:00:00,116.493,203.043,2024-11-25 18:22:20.657
2024-11-25 23:00:00,99.662,160.490,2024-11-25 18:22:20.657
2024-11-26 00:00:00,148.664,152.849,2024-11-26 18:22:21.843
2024-11-26 01:00:00,279.199,170.416,2024-11-26 18:22:21.843
2024-11-26 02:00:00,178.180,237.368,2024-11-26 18:22:21.847


In [10]:
mi3_volumes = volumes[volumes['MARKET'] == 'MI3']
mi3_volumes = mi3_volumes.drop(columns=['MARKET'])
mi3_volumes_nord = mi3_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi3_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-11-24 18:00:00,149.671,187.538,2024-11-24 18:22:35.160
2024-11-24 19:00:00,91.193,118.713,2024-11-24 18:22:35.160
2024-11-24 20:00:00,131.033,123.911,2024-11-24 18:22:35.160
2024-11-24 21:00:00,93.644,146.344,2024-11-24 18:22:35.160
2024-11-24 22:00:00,57.802,121.580,2024-11-24 18:22:35.160
2024-11-24 23:00:00,173.571,158.216,2024-11-24 18:22:35.160
2024-11-25 12:00:00,347.465,506.811,2024-11-25 18:22:23.777
2024-11-25 13:00:00,208.110,483.048,2024-11-25 18:22:23.777
2024-11-25 14:00:00,306.388,373.138,2024-11-25 18:22:23.777


In [11]:
mi4_volumes = volumes[volumes['MARKET'] == 'MI4']
mi4_volumes = mi4_volumes.drop(columns=['MARKET'])
mi4_volumes_nord = mi4_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi4_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2021-09-19 10:00:00,35.446,335.960,2021-09-19 04:10:57.840
2021-09-19 11:00:00,27.397,48.069,2021-09-19 04:10:57.840
2021-09-19 12:00:00,13.714,41.316,2021-09-19 04:10:57.843
2021-09-19 13:00:00,20.175,12.943,2021-09-19 04:10:57.843
2021-09-19 14:00:00,27.808,58.166,2021-09-19 04:10:57.843
2021-09-19 15:00:00,17.959,10.590,2021-09-19 04:10:57.843
2021-09-19 16:00:00,42.771,94.523,2021-09-19 04:10:57.843
2021-09-19 17:00:00,110.805,244.656,2021-09-19 04:10:57.843
2021-09-19 18:00:00,120.737,350.911,2021-09-19 04:10:57.843
